# 🫁 Pneumothorax AI — Global Pre-training (Kaggle)
**TÜBİTAK 2209-A | Ahmet Demir | Dokuz Eylül Üniversitesi**

**Veri:** SIIM-ACR Pneumothorax Segmentation (~12 GB, piksel maskeli)

**Gereksinimler:**
- Settings → Accelerator: **GPU P100**
- Input: **SIIM-ACR Pneumothorax Segmentation** (competition dataset)
- Internet: **ON**

## 0. GPU Kontrolü

In [ ]:
import torch, os, subprocess
print('GPU :', torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'YOK — Settings > Accelerator > GPU P100 seç!')
print('CUDA:', torch.version.cuda)
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

## 1. SIIM Veri Yolunu Kontrol Et

In [ ]:
# SIIM competition verisi /kaggle/input/ altında olmalı
import glob

# Olası SIIM input yolları
SIIM_INPUT = None
candidates = [
    '/kaggle/input/siim-acr-pneumothorax-segmentation',
    '/kaggle/input/siim-acr-pneumothorax-segmentation-data',
    '/kaggle/input/siim-acr-pneumothorax',
]
for c in candidates:
    if os.path.exists(c):
        SIIM_INPUT = c
        break

if SIIM_INPUT:
    dcm_count = int(subprocess.check_output(
        f'find {SIIM_INPUT} -name "*.dcm" | wc -l', shell=True
    ).decode().strip())
    print(f'✓ SIIM bulundu: {SIIM_INPUT}')
    print(f'  DICOM sayısı: {dcm_count:,}')
    print(f'  İçerik: {os.listdir(SIIM_INPUT)[:8]}')
else:
    print('[!] SIIM Input bulunamadı!')
    print('    → Sağ panelden Add Input → SIIM-ACR Pneumothorax Segmentation')
    print('    Mevcut inputlar:', os.listdir('/kaggle/input/') if os.path.exists('/kaggle/input') else 'yok')

## 2. Projeyi Klonla

In [ ]:
REPO = 'https://github.com/ahmetai-cell/pneumothorax-ai-detection'
PROJECT_DIR = '/kaggle/working/pneumothorax-ai-detection'

if os.path.exists(PROJECT_DIR):
    !cd {PROJECT_DIR} && git pull
else:
    !git clone {REPO}

os.chdir(PROJECT_DIR)
print('Çalışma dizini:', os.getcwd())

## 3. Bağımlılıkları Yükle

In [ ]:
!pip install -q \
    segmentation-models-pytorch \
    albumentations \
    pydicom \
    pynrrd \
    wandb \
    tqdm \
    fpdf2 \
    plotly
print('✓ Kurulum tamamlandı')

## 4. SIIM Verisini Proje Dizinine Bağla

In [ ]:
import os, glob, shutil

os.makedirs('data/raw/global', exist_ok=True)
os.makedirs('data/processed', exist_ok=True)
os.makedirs('data/masks/siim', exist_ok=True)
os.makedirs('checkpoints', exist_ok=True)
os.makedirs('results', exist_ok=True)

SIIM_LOCAL = 'data/raw/global/siim'

# SIIM input yolunu bul
SIIM_INPUT = None
for c in [
    '/kaggle/input/siim-acr-pneumothorax-segmentation',
    '/kaggle/input/siim-acr-pneumothorax-segmentation-data',
    '/kaggle/input/siim-acr-pneumothorax',
]:
    if os.path.exists(c):
        SIIM_INPUT = c
        break

if SIIM_INPUT:
    if os.path.islink(SIIM_LOCAL):
        os.unlink(SIIM_LOCAL)
    if not os.path.exists(SIIM_LOCAL):
        os.symlink(SIIM_INPUT, SIIM_LOCAL)
        print(f'✓ Symlink: {SIIM_LOCAL} → {SIIM_INPUT}')
    else:
        print(f'✓ Zaten mevcut: {SIIM_LOCAL}')
    
    # DICOM sayısı
    dcm_count = int(subprocess.check_output(
        f'find {SIIM_INPUT} -name "*.dcm" | wc -l', shell=True
    ).decode().strip())
    print(f'  DICOM: {dcm_count:,}')
    
    # train-rle.csv varlığını kontrol et
    rle_candidates = glob.glob(f'{SIIM_INPUT}/*rle*.csv') + glob.glob(f'{SIIM_INPUT}/**/*rle*.csv')
    if rle_candidates:
        print(f'  RLE CSV: {rle_candidates[0]}')
    else:
        print('  [!] RLE CSV bulunamadı — içerik:', os.listdir(SIIM_INPUT)[:10])
else:
    print('[!] SIIM Input eklenmemiş!')
    print('    Mevcut inputlar:', os.listdir('/kaggle/input/'))

## 5. RLE → Maske Dönüştür & Manifest Oluştur

In [ ]:
# SIIM RLE maskelerini NRRD'ye dönüştür
!python scripts/data_manager.py --convert_rle
# Master manifest oluştur
!python scripts/data_manager.py --build_manifest

import pandas as pd
try:
    df = pd.read_csv('data/processed/master_manifest.csv')
    print(f'\n✓ Manifest: {len(df):,} kayıt')
    print(f'  Pozitif : {(df["is_pneumo"]==1).sum():,}')
    print(f'  Negatif : {(df["is_pneumo"]==0).sum():,}')
    print(f'  Kaynaklar: {df["source"].value_counts().to_dict()}')
except Exception as e:
    print(f'[!] {e}')

## 6. W&B Giriş

In [ ]:
WANDB_KEY = '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'

if WANDB_KEY:
    import wandb
    wandb.login(key=WANDB_KEY, relogin=True)
    USE_WANDB = True
    print('✓ W&B tamam')
else:
    USE_WANDB = False
    print('W&B atlandı')

## 7. Pre-training Başlat

**Kaggle P100'de tahmini süre:**
- SIIM ~12k görüntü, batch=16 → 1 epoch ~2 dk
- 50 epoch ≈ ~1.5 saat

> Checkpoint her fold'dan sonra kaydedilir.

In [ ]:
WANDB_FLAG = '' if USE_WANDB else '--no_wandb'
CKPT_DIR   = '/kaggle/working/checkpoints'
os.makedirs(CKPT_DIR, exist_ok=True)

!python scripts/train_global.py \
    --sources SIIM \
    --encoder efficientnet-b0 \
    --img_size 512 \
    --epochs 50 \
    --batch_size 16 \
    --num_folds 5 \
    --lr 1e-4 \
    {WANDB_FLAG} \
    --checkpoint_dir {CKPT_DIR}

## 8. Sonuçlar

In [ ]:
import pandas as pd, json, glob as gl

for p in [f'{CKPT_DIR}/global_kfold_results.csv', 'results/global_kfold_results.csv']:
    if os.path.exists(p):
        df = pd.read_csv(p)
        print('=== K-FOLD SONUÇLARI ===')
        print(df.to_string(index=False))
        print(f'\nOrtalama Dice : {df["best_dice"].mean():.4f} ± {df["best_dice"].std():.4f}')
        print(f'Ortalama AUC  : {df["best_auc"].mean():.4f} ± {df["best_auc"].std():.4f}')
        break
else:
    print('Results CSV bulunamadı')

for mp in [f'{CKPT_DIR}/global_base_model_meta.json']:
    if os.path.exists(mp):
        meta = json.load(open(mp))
        print('\n=== BASE MODEL ===')
        for k, v in meta.items(): print(f'  {k}: {v}')
        break

ckpts = gl.glob(f'{CKPT_DIR}/**/*.pth', recursive=True)
print(f'\n  Checkpoint sayısı: {len(ckpts)}')
for c in sorted(ckpts):
    print(f'  {os.path.basename(c):40s}  {os.path.getsize(c)/1e6:.1f} MB')

## 9. Checkpoint'leri Output'a Kopyala

In [ ]:
import shutil
OUTPUT = '/kaggle/working'

for src, dst in [
    (f'{CKPT_DIR}/global_base_model.pth',      f'{OUTPUT}/global_base_model.pth'),
    (f'{CKPT_DIR}/global_base_model_meta.json', f'{OUTPUT}/global_base_model_meta.json'),
    (f'{CKPT_DIR}/global_kfold_results.csv',    f'{OUTPUT}/global_kfold_results.csv'),
]:
    if os.path.exists(src):
        shutil.copy(src, dst)
        print(f'✓ {os.path.basename(src)}')

for ckpt in gl.glob(f'{CKPT_DIR}/global_folds/*.pth'):
    shutil.copy(ckpt, f'{OUTPUT}/{os.path.basename(ckpt)}')
    print(f'✓ {os.path.basename(ckpt)}')

print('\n✓ Output sekmesinden indirebilirsin')